In [ ]:
wien_lamFlam_x = 3.90269

wien_lamFlam_factor = (h*c)/(wien_lamFlam_x*k)

def disk_fitter(centwavs, logTdisk, logxdisk):
    centwavs_angstroms = centwavs * 1.e10 * u.AA
    centwavs_meters = centwavs * u.m
    
    Tdisk = 10.**logTdisk
    xdisk = 10.**logxdisk
    
    #print centwavs_angstroms
    #print Teff
    
    #print blackbody_lambda(centwavs_angstroms[0], Teff*u.K)
    
    #flux_lam_temp = []
    #for wav in centwavs_angstroms:
    #    flux_lam_temp.append(blackbody_lambda(wav, Teff*u.K) * np.pi * u.sr)
    
    flux_lam_temp = blackbody_lambda(centwavs_angstroms, Tdisk*u.K) * np.pi * u.sr 
    
    #print flux_lam_temp
    
    flux_lam = flux_lam_temp * centwavs_angstroms * u.cm * u.cm * u.s / u.erg
    
    #print flux_lam 
    
    #print Tdisk, xdisk, max(flux_lam)
    
    #print xdisk
    
    return flux_lam * xdisk
    
def get_objective_function_value(subj, logTdiskguess, logxdiskguess):
    #Tdiskguess, xdiskguess = params
    #logTdiskguess = np.log10(Tdiskguess)
    #logxdiskguess = np.log10(xdiskguess)
    flux_lam_fit_disk = disk_fitter(subj.centwavs_meters, logTdiskguess, logxdiskguess)
    
    flux_lam_fit = flux_lam_fit_disk + subj.nuFnu_star
    
    flux_lam = subj.nuFnu
    flux_lam_errs = subj.nuFnuerrs
    
    #print min(flux_lam_errs)
    
    chiconts = np.zeros(flux_lam.size)
    
    for i in range(flux_lam.size):
        chiconts[i] = ((flux_lam[i] - flux_lam_fit[i])/flux_lam_errs[i])**2
        
    chisum = np.sum(chiconts)
    
    return chisum
        

def get_disk_fit(subj):
    subj.good_disk_fit = False
    nuFnu_use = subj.nuFnu_disk[-4:]
    nuFnu_err_use = subj.nuFnu_disk_errs[-4:]
    cent_wavs_use = subj.centwavs_meters[-4:]
    
    #print nuFnu_use
    #print nuFnu_err_use
    #print cent_wavs_use
    
    #Tstar = subj.Teff
    #rdstar = subj.rdstar
    
    #sig_nuFnu_use = nuFnu_use / nuFnu_err_use
    #print sig_nuFnu_use
    #print sig_nuFnu_use[sig_nuFnu_use > 5.]
    #print sig_nuFnu_use[sig_nuFnu_use > 5.].size
    
    if subj.num_excesses < 1:
        print "No significant excess"
        subj.nuFnu_model_plotting = subj.nuFnu_star_plotting
        return subj
    elif subj.num_excesses > 1:
        Teff_use = 10.**subj.log10Teffguess
        rdstar_use = 10.**subj.log10rdstarguess
        
        Tdiskguess = 200.
        xdiskguess = 1.e-3 * ((Teff_use/Tdiskguess)**4)*(rdstar_use**2)
        guess = [np.log10(200.), np.log10(xdiskguess)]
        popt, pcov = curve_fit(disk_fitter, cent_wavs_use, nuFnu_use, p0 = guess, sigma = nuFnu_err_use)
        
        subj.Tdisk = 10.**(popt[0])
        subj.xdisk = 10.**(popt[1])
        subj.fir = ((subj.Tdisk/subj.Teff)**4)*(subj.xdisk/(subj.rdstar**2))
        
        print "Excesses:", subj.num_excesses
        nuFnu_disk_points = blackbody_lambda(subj.centwavs_meters * 1.e10 * u.AA, subj.Tdisk * u.K) * np.pi * u.sr * (subj.centwavs_meters * 1.e10 * u.AA) * subj.xdisk * u.cm * u.cm * u.s / u.erg
        print nuFnu_disk_points
        print subj.nuFnu_disk
        
        subj.good_disk_fit = True
        
        subj.nuFnu_disk_plotting = blackbody_lambda(plotting_xvec_angstroms * u.AA, subj.Tdisk*u.K) * np.pi * u.sr * (plotting_xvec_angstroms * u.AA) * subj.xdisk * u.cm * u.cm * u.s / u.erg
        subj.nuFnu_model_plotting = subj.nuFnu_star_plotting  + subj.nuFnu_disk_plotting
        
        return subj
    
    else:
        print "Excesses:", subj.num_excesses
        #while not subj.good_disk_fit:
        #lambda_max_star = (2.8977729e-3 / Tstar)

        #peak_flux_star = blackbody_lambda((lambda_max_star*1.e10*u.AA), (Tstar*u.K))

        #lambda_max_disk = (2.8977729e-3 / 1.)
    
        #test_min_peak_flux_disk = blackbody_lambda((lambda_max_disk*1.e10*u.AA), (1.*u.K))
        #test_max_peak_flux_disk = blackbody_lambda(((2.8977729e-3/500.)*1.e10*u.AA), (500.*u.K))
    
        #test_min_peak_flux_disk_at_star = blackbody_lambda((lambda_max_star*1.e10*u.AA), (1.*u.K))
        #test_max_peak_flux_disk_at_star = blackbody_lambda(((lambda_max_star*1.e10*u.AA), (500.*u.K)))
    
        #test_min_peak_flux_star_at_disk = blackbody_lambda((lambda_max_disk*1.e10*u.AA), (Tstar*u.K))
        #test_max_peak_flux_star_at_disk = blackbody_lambda(((2.8977729e-3/500.)*1.e10*u.AA), (Tstar*u.K))
    
        #xdiskmax = 0.1 * rdstar * ((lambda_max_star * peak_flux_star) - (lambda_max_disk * test_min_peak_flux_star_at_disk))/((lambda_max_disk * test_min_peak_flux_disk) - (lambda_max_star * test_min_peak_flux_disk_at_star))
        #xdiskmin = 0.1* float((peak_flux_star / test_max_peak_flux_disk)) * (lambda_max_star/(2.8977729e-3/500.)) * rdstar
    
        #Tdiskmax = 0.1*Tstar
        #xdiskmax = 2.e14*rdstar
        #xdiskmin = float(1.e9*rdstar)
    
        #print xdiskmin, xdiskmax
    
        Tdiskvals = np.linspace(25., 300., 12)
        #xdiskvals = np.linspace(xdiskmin, 1.e11*rdstar, 51)
    
        #valgrid = np.zeros(51*51).reshape(51,51)
        #reschi = np.zeros(51*51).reshape(51,51)
    
        val_chi_dict = {}
        
        xdiskvals_dict = {}
        
        Tdiskval_minchis_dict = {}
    
        for i in range(12):
            Tdiskval = Tdiskvals[i]
        
            xdiskmax = (subj.rdstar**2) * 1.e-2 * ((subj.Teff/Tdiskval)**4)
            xdiskmin = (subj.rdstar**2) * 1.e-9 * ((subj.Teff/Tdiskval)**4)
         
            xdiskvals = np.linspace(xdiskmin, xdiskmax, 51)
            
            xdiskvals_dict[Tdiskval] = xdiskvals
            
            for j in range(51):
                val_chi_dict[(Tdiskvals[i], xdiskvals[j])] = get_objective_function_value(subj, np.log10(Tdiskvals[i]), np.log10(xdiskvals[j]))
                #valgrid[i,j] = (Tdiskvals[i], xdiskvals[j])
                #reschi[i,j] = get_objective_function_value(subj, Tdiskvals[i], xdiskvals[j])
    
        minchi = np.inf
        minchi_vals = None
   
        possible_minchi_vals = []
    
        minchi_chis_by_temperature = {Temp: np.inf for Temp in Tdiskvals}
        minchi_vals_by_temperature = {Temp: None for Temp in Tdiskvals}
    
        for key in val_chi_dict.keys():
            keytemp = key[0]
            
            if val_chi_dict[key] < minchi_chis_by_temperature[keytemp]:
                minchi_chis_by_temperature[keytemp] = val_chi_dict[key]
                minchi_vals_by_temperature[keytemp] = key
            
            if val_chi_dict[key] < minchi:
                minchi = val_chi_dict[key]
                minchi_vals = key

        for key in val_chi_dict.keys():
            local_min = True
            if (val_chi_dict[key] - minchi)/(minchi) < 1.:
                Tdiskval = key[0]
                xdiskval = key[1]
                xdiskindex = np.where(xdiskvals_dict[Tdiskval] == xdiskval)
                Tdiskindex = np.where(Tdiskvals==Tdiskval)
                
                if (Tdiskindex[0] > 0) and (Tdiskindex[0] < 11) and (xdiskindex[0] > 0) and (xdiskindex[0] < 50):
                    for i in range(Tdiskindex[0] - 1, Tdiskindex[0] + 2):
                        xdiskvals_surrounding = xdiskvals_dict[Tdiskvals[i]]
                        for j in range(xdiskindex[0] - 1, xdiskindex[0] + 2):
                            if (val_chi_dict[key] > val_chi_dict[(Tdiskvals[i], xdiskvals_surrounding[j])]):
                                local_min = False
                                
                if local_min:
                    possible_minchi_vals.append(key)
                
    #from pprint import pprint
    #pprint(val_chi_dict)
                
        print minchi_vals
        print len(possible_minchi_vals)
    
        possible_minchi_results_dict = {}
        possible_minchi_chi_dict = {}
    
        for possible_minchi_val in possible_minchi_vals:
            guess = np.array([np.log10(possible_minchi_val[0]), np.log10(possible_minchi_val[1])])
    
            #print nuFnu_use
            try:
                popt, pcov = curve_fit(disk_fitter, cent_wavs_use, nuFnu_use, guess, sigma = nuFnu_err_use)
            except RuntimeError:
                continue
            possible_minchi_results_dict[possible_minchi_val] = np.array(popt)
            possible_minchi_chi_dict[possible_minchi_val] = get_objective_function_value(subj, popt[0], popt[1])
    
        #final_minchi = np.inf
        #final_minchi_vals = None
        
        minchi_temp_results_dict = {}
        minchi_temp_chi_dict = {}
        
        for minchi_val_by_temp in minchi_vals_by_temperature.values():
            guess = np.array([np.log10(minchi_val_by_temp[0]), np.log10(minchi_val_by_temp[1])])
            
            try:
                popt, pcov = curve_fit(disk_fitter, cent_wavs_use, nuFnu_use, guess, sigma = nuFnu_err_use)
            except RuntimeError:
                continue
                
            minchi_temp_results_dict[minchi_val_by_temp] = np.array(popt)
            minchi_temp_chi_dict[minchi_val_by_temp] = get_objective_function_value(subj, popt[0], popt[1])
            
        sorted_chis = sorted(possible_minchi_chi_dict.values())
        
        print len(sorted_chis)
        
        sorted_minchi_keys = []
        
        for chi in sorted_chis:
            sorted_minchi_keys.append(possible_minchi_chi_dict.keys()[possible_minchi_chi_dict.values().index(chi)])
        
        sorted_results = [possible_minchi_results_dict[x] for x in sorted_minchi_keys]
        
        sorted_temp_chis = sorted(minchi_temp_chi_dict.values())
        
        print len(sorted_temp_chis)
        
        sorted_temp_minchi_keys = []
        
        for chi in sorted_temp_chis:
            sorted_temp_minchi_keys.append(minchi_temp_chi_dict.keys()[minchi_temp_chi_dict.values().index(chi)])
            
        sorted_temp_results = [minchi_temp_results_dict[x] for x in sorted_temp_minchi_keys]
        
        result_index = 0
        result_index_temp = 0
        while not subj.good_disk_fit:
            if result_index > (len(sorted_results)-1):
                print "Out of values; testing temperature results"
                
                result = sorted_temp_results[result_index_temp]
                
                print result
                print 10.**result
                test_Tdisk = 10.**(result[0])
                test_xdisk = 10.**(result[1])
                
                test_fir = ((test_Tdisk/subj.Teff)**4) * (test_xdisk/(subj.rdstar**2))
                
                peak_lambda_disk = wien_lamFlam_factor / test_Tdisk
                
                peak_lamFlam_disk = None
                
                if peak_lambda_disk < (100.*1.e-6):
                    peak_lamFlam_disk = blackbody_lambda(peak_lambda_disk * 1.e10 * u.AA, test_Tdisk * u.K) * np.pi * u.sr * (peak_lambda_disk * 1.e10 * u.AA) * test_xdisk * u.cm * u.cm * u.s / u.erg
                else:
                    peak_lamFlam_disk = blackbody_lambda(100. * 1.e4 * u.AA, test_Tdisk * u.K) * np.pi * u.sr * (100. * 1.e4 * u.AA) * test_xdisk * u.cm * u.cm * u.s / u.erg
    
                peak_lamFlam_star = None
        
                if np.argmax(subj.nuFnu_star_plotting) > 0:
                    peak_lamFlam_star = max(subj.nuFnu_star_plotting)
                else:
                    peak_lambda_star = wien_lamFlam_factor / subj.Teff
                    peak_lamFlam_star = blackbody_lambda(peak_lambda_star * 1.e10 * u.AA, subj.Teff * u.K) * np.pi * u.sr * (peak_lambda_star * 1.e10 * u.AA) * (subj.rdstar**2) * u.cm * u.cm * u.s / u.erg
        
                if (test_fir < 0.01) and (peak_lamFlam_disk < peak_lamFlam_star):
                    subj.Tdisk = test_Tdisk
                    subj.xdisk = test_xdisk
                    subj.fir = test_fir
                    subj.good_disk_fit = True
                    print "Good fit"
                else:
                    print "Fit fail--retrying"
              
                result_index_temp += 1
                
                if result_index_temp > (len(sorted_temp_results) - 1):
                    print "Out of trial values, giving up"
                    break

            else:
                #if (possible_minchi_chi_dict[key] < final_minchi): #and (peak_lamFlam_disk < peak_lamFlam_star):
                #    final_minchi = possible_minchi_chi_dict[key]
                #    final_minchi_vals = possible_minchi_results_dict[key]
                result = sorted_chi_results[result_index]
                                      
                print result
                print 10.**result
                test_Tdisk = 10.**(result[0])
                #subj.Tdisk_err = (pcov[0,0])**2
                test_xdisk = 10.**(result[1])
                #subj.xdisk_err = (pcov[1,1])**2
    
                #print subj.Tdisk, subj.xdisk
                test_fir = ((test_Tdisk/subj.Teff)**4)*(test_xdisk/(subj.rdstar**2))
        
                peak_lambda_disk = wien_lamFlam_factor / test_Tdisk
         
                peak_lamFlam_disk = None
    
                if peak_lambda_disk < (100.*1.e-6):
                    peak_lamFlam_disk = blackbody_lambda(peak_lambda_disk * 1.e10 * u.AA, test_Tdisk * u.K) * np.pi * u.sr * (peak_lambda_disk * 1.e10 * u.AA) * test_xdisk * u.cm * u.cm * u.s / u.erg
                else:
                    peak_lamFlam_disk = blackbody_lambda(100. * 1.e4 * u.AA, test_Tdisk * u.K) * np.pi * u.sr * (100. * 1.e4 * u.AA) * test_xdisk * u.cm * u.cm * u.s / u.erg
        
                peak_lamFlam_star = None
        
                if np.argmax(subj.nuFnu_star_plotting) > 0:
                    peak_lamFlam_star = max(subj.nuFnu_star_plotting)
                else:
                    peak_lambda_star = wien_lamFlam_factor / subj.Teff
                    peak_lamFlam_star = blackbody_lambda(peak_lambda_star * 1.e10 * u.AA, subj.Teff * u.K) * np.pi * u.sr * (peak_lambda_star * 1.e10 * u.AA) * (subj.rdstar**2) * u.cm * u.cm * u.s / u.erg
        
                if (test_fir < 0.01) and (peak_lamFlam_disk < peak_lamFlam_star):
                    subj.Tdisk = test_Tdisk
                    subj.xdisk = test_xdisk
                    subj.fir = test_fir
                    subj.good_disk_fit = True
                    print "Good fit"
                else:
                    print "Fit fail--retrying"
            
                result_index += 1
            
        if subj.good_disk_fit:
            nuFnu_disk_points = blackbody_lambda(subj.centwavs_meters * 1.e10 * u.AA, subj.Tdisk * u.K) * np.pi * u.sr * (subj.centwavs_meters * 1.e10 * u.AA) * subj.xdisk * u.cm * u.cm * u.s / u.erg
            print nuFnu_disk_points
            print subj.nuFnu_disk
            #nuFnu_disk_points = blackbody_lambda(cent_wavs_use*1.e10 *u.AA, subj.Tdisk * u)
        
            subj.nuFnu_disk_plotting = blackbody_lambda(plotting_xvec_angstroms * u.AA, subj.Tdisk*u.K) * np.pi * u.sr * (plotting_xvec_angstroms * u.AA) * subj.xdisk * u.cm * u.cm * u.s / u.erg
            print np.interp(subj.centwavs_meters*1.e10, plotting_xvec_angstroms, subj.nuFnu_disk_plotting)
        
            subj.nuFnu_model_plotting = subj.nuFnu_star_plotting + subj.nuFnu_disk_plotting
            #subj.nuFnu_disk = subj.nuFnu - subj.nuFnu_star
    
            #subj.nuFnu_disk_errs = subj.nuFnuerrs
    
            return subj
    
        else:
            subj.nuFnu_model_plotting = subj.nuFnu_star_plotting
            return subj


#subj_test = subjs_with_star_fits[40]

subjs_with_disk_fits = []

plt.close("all")

disk_fit_start_time = time.time()

subj_counter = 0

for subj in subjs_with_star_fits:
    print subj.zooniverse_id, subj.wiseid
    
    subj_counter += 1
    
    if not subj.good_star_fit:
        print "No good star fit"
        continue
    else:
        print subj.num_excesses
        subjs_with_disk_fits.append(get_disk_fit(subj))
        print subj.Tdisk, subj.xdisk
    
    plot_subj(subj)

    #if subj.num_excesses > 1:
    #    f1.write(subj.zooniverse_id+','+subj.wiseid+'\n')
    #elif subj.num_excesses < 1:
    #    f2.write(subj.zooniverse_id+','+subj.wiseid+'\n')
    #else:
    #    f3.write(subj.zooniverse_id+','+subj.wiseid+'\n')
    
    print len(subjs_with_disk_fits), len(subjs_with_star_fits), (time.time() - disk_fit_start_time)/len(subjs_with_disk_fits)
    
disk_fit_end_time = time.time()

#f1.close()
#f2.close()
#f3.close()
    
print "Disk fits done"
print "Time elapsed:", disk_fit_end_time - disk_fit_start_time

